In [2]:
import torch
import torch.nn as nn
import cv2
from model.resnet import ResNet50, ResNet101, ResNet152

model = ResNet101(num_classes=10, channels=1).cuda()
model.load_state_dict(torch.load('./model_99.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (i_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, m

In [5]:
import os
from tool import *
from tqdm import tqdm
# img = cv2.imread('./frames/s03_cond3_2/frame_0322.jpg', cv2.IMREAD_GRAYSCALE)

path = './frames/s02_cond3_2'
output_path = './output/s02_cond3_2'
if not os.path.exists(output_path):
    os.makedirs(output_path)
for file in tqdm(os.listdir(path)):
    img = cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE)
    height_extension = 100
    new_height = img.shape[0] + height_extension
    new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
    new_image[:img.shape[0], :] = img
    img = cv2.resize((new_image/255)*2 - 1, (512, 512))
    input_img = torch.tensor(img).float().cuda()
    input_img = input_img.unsqueeze(0).unsqueeze(0)
    output = model(input_img)
    output = output.squeeze().cpu().detach().numpy()
    result = draw_img(img, output)
    cv2.imwrite(os.path.join(output_path, file), result)

# height_extension = 100
# new_height = img.shape[0] + height_extension
# new_image = np.zeros((new_height, img.shape[1]), dtype=np.float32)
# new_image[:img.shape[0], :] = img
# img = cv2.resize((new_image/255)*2 - 1, (512, 512))
# input_img = torch.tensor(img).float().cuda()
# input_img = input_img.unsqueeze(0).unsqueeze(0)
# print(input_img.shape)

100%|██████████| 478/478 [00:26<00:00, 17.78it/s]


In [37]:
from tool import *
output = model(input_img)
print(output.shape)

output = output.squeeze().cpu().detach().numpy()
result = draw_img(img, output)
cv2.imwrite('result.png', result)

torch.Size([1, 10])
(600, 600)


True